In [ ]:
import re
import mysql.connector
import string
import time
import random
from datetime import date

def result(username):
    cursor.execute("SELECT * FROM quiz_results where username = %s", (username,))
    quiz_result = cursor.fetchall()
    
    for i in quiz_result:
        username, quiz_table, today, score, total_questions = i
        
        print("your result on", quiz_table, today , "was:", score,"/", total_questions)

def fetch_random_questions(table_name,num_questions=3):
        # Fetch random questions from the database
        cursor.execute("SELECT * FROM {} ORDER BY RAND() LIMIT %s".format(table_name), (num_questions,))
        questions = cursor.fetchall()
        return questions

def run_quiz(questions, quiz_table, total_questions):
        score = 0
        today = date.today()

        for question in questions:
            q_id, q_text, q_options, q_answer = question

            print(f"\n{q_text}")
            print(f"\n{q_options}")

            user_answer = (input("Your answer: "))

            if user_answer == q_answer:
                print("Correct!")
                score += 1
            else:
                print("Incorrect. The correct answer is {}".format(q_answer))

        print("\nQuiz completed. Your result on",today, "is: {}/{}".format(score, len(questions)))
        
        cursor.execute("INSERT INTO quiz_results (username, quiz_table, today, score, total_questions) VALUES (%s, %s, %s, %s, %s)",
                       (username, quiz_table, today, score, total_questions))
        
        login_page()
        

def quiz():
        print("Select Language: \n1.C++ \n2.Python \n3.Java")
        x = int(input())
        if x==1:
            # Specify the table name for the quiz
            quiz_table = "CPP_questions"
            n = int(input("Enter the number of questions you want to attempt: "))
            random_questions = fetch_random_questions(quiz_table, n)

            # Run the quiz with the random questions
            run_quiz(random_questions, quiz_table, n)
            
        elif x==2:
            quiz_table = "Python_questions"
            n = int(input("Enter the number of questions you want to attempt: "))
            random_questions = fetch_random_questions(quiz_table, n)
            run_quiz(random_questions, quiz_table, n)
            
        elif x==3:
            quiz_table = "JAVA_questions"
            n = int(input("Enter the number of questions you want to attempt: "))
            random_questions = fetch_random_questions(quiz_table, n)
            run_quiz(random_questions, quiz_table, n)
            
        else:
            print('\n Wrong Input. Try Again')
            quiz()

def profile(username):
    try:
        cursor.execute("SELECT * FROM users WHERE username = %s", (username,))
        row = cursor.fetchone()

        if row:
            username, password, name, college, roll = row
            
            print("username = {}, password = {}, name = {}, college = {}, roll = {}".format(username, password, name, college, roll))
        else:
            print("User not found.")
            
    except mysql.connector.Error as err:
        print("Error: {}".format(err))

def login_page():
    print('Choose:\n1.Attempt quiz \n2.Previous Result \n3.See profile \n4.Logout')
    x = int(input())
    
    if x==1:
        quiz()
    elif x==2:
        result(username)
    elif x==3:
        profile(username)
    elif x==4:
        exit()
    else:
        print('\n Wrong Input. Try Again')
        login_page()
    return

def is_password_valid(password):
    # Check if the password meets the specified criteria
    pattern = re.compile(r'^(?=.*[a-z])(?=.*[A-Z])(?=.*\d)(?=.*[@$!%*?&])[A-Za-z\d@$!%*?&]{8,20}$')
    return bool(re.match(pattern, password))

def register_user(cursor, connection):
    print("User Registration:")
    username = input("Enter username: ")
    password = input("Enter password: ")
    name = input("Enter your name:")
    college = input("Enter your college:")
    roll = input("Enter your Enrollment number:")

    if is_password_valid(password):
        try:
            # Check if the username is unique
            cursor.execute("SELECT * FROM users WHERE username = %s", (username,))
            if cursor.fetchone():
                print("Username already exists. Please choose a different username.")
            else:
                # Insert new user into the database
                cursor.execute("INSERT INTO users (username, password, name, college, roll) VALUES (%s, %s, %s, %s, %s)", (username, password, name, college, roll))
                connection.commit()
                print("Registration successful.")
        except mysql.connector.Error as err:
            print(f"Error: {err}")
    else:
        print("Invalid password format. Password should be 8-20 characters with 1 uppercase, 1 lowercase, 1 numeric, and 1 special character.")
    

def login_user(cursor, connection):
    print("User Login:")
    global username
    username = input("Enter username: ")
    password = input("Enter password: ")

    try:
        # Retrieve user from the database based on username and password
        cursor.execute("SELECT * FROM users WHERE username = %s AND password = %s", (username, password))
        user = cursor.fetchone()

        if user:
            print("Login successful. \n \n")
            login_page()
        else:
            print("Invalid username or password.")
    except mysql.connector.Error as err:
        print(f"Error: {err}")
    

def main():
    # Connect to the MySQL database
    global connection
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="12345678",
        database="PROFILE"
    )

    if connection.is_connected():
        global cursor
        cursor = connection.cursor()

        # Create a 'users' table if it doesn't exist
        cursor.execute("CREATE TABLE IF NOT EXISTS users (username VARCHAR(25) PRIMARY KEY, password VARCHAR(20) NOT NULL, name varchar(30),college varchar(50), roll varchar(20), Result INT )")

        while True:
            print("\n1. Register\n2. Login\n3. Exit")
            choice = input("Enter your choice (1/2/3): ")

            if choice == '1':
                register_user(cursor, connection)
            elif choice == '2':
                login_user(cursor, connection)
            elif choice == '3':
                break
            else:
                print("Invalid choice. Please enter 1, 2, or 3.")

        # Close the cursor and connection when done
        cursor.close()
        connection.close()
        print("Connection closed.")

if __name__ == "__main__":
    main()



1. Register
2. Login
3. Exit
Enter your choice (1/2/3): 2
User Login:
Enter username: varsha_trivedi
Enter password: Varsha@123
Login successful. 
 

Choose:
1.Attempt quiz 
2.Previous Result 
3.See profile 
4.Logout
3
username = varsha_trivedi, password = Varsha@123, name = Varsha Rani, college = LNCTU, roll = LNCBBTAIM060

1. Register
2. Login
3. Exit
